# Genomic Pleiotropy Analysis Notebook

This interactive notebook provides tools for exploring and analyzing genomic pleiotropy data using the Python analysis modules and Rust core functionality.

## 1. Setup and Imports

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Import our custom modules
from trait_visualizer import TraitVisualizer
from statistical_analyzer import StatisticalAnalyzer
from rust_interface import RustInterface, InterfaceMode, dataframe_to_trait_data, results_to_dataframe

print("All modules imported successfully!")

## 2. Initialize Components

In [ ]:
# Initialize the analysis components
visualizer = TraitVisualizer(style='seaborn-v0_8')
analyzer = StatisticalAnalyzer(multiple_testing_method='fdr_bh')

# Initialize Rust interface (subprocess mode by default)
# Change to InterfaceMode.PYO3 if you have built the PyO3 extension
rust_interface = RustInterface(mode=InterfaceMode.SUBPROCESS)

print("Components initialized!")
print(f"Rust core version: {rust_interface.get_version()}")

## 3. Load and Decrypt Trait Data

In [ ]:
# Example: Load encrypted trait data
# Replace with your actual file paths
encrypted_file = "../data/encrypted_traits.dat"
key_file = "../data/trait_key.key"

# Decrypt and load trait data
try:
    trait_df = rust_interface.decrypt_trait_data(
        encrypted_file=encrypted_file,
        key_file=key_file,
        output_format='dataframe'
    )
    print(f"Loaded {trait_df.shape[0]} samples with {trait_df.shape[1]} traits")
    print("\nTrait names:")
    print(list(trait_df.columns))
except Exception as e:
    print(f"Error loading encrypted data: {e}")
    print("Generating synthetic data for demonstration...")
    
    # Generate synthetic trait data for demonstration
    np.random.seed(42)
    n_samples = 1000
    n_traits = 10
    
    # Create correlated traits
    trait_names = [f"Trait_{i+1}" for i in range(n_traits)]
    
    # Generate correlation structure
    base_data = np.random.randn(n_samples, n_traits)
    
    # Add correlations between some traits
    base_data[:, 1] = base_data[:, 0] * 0.7 + np.random.randn(n_samples) * 0.3
    base_data[:, 2] = base_data[:, 0] * 0.5 + base_data[:, 1] * 0.3 + np.random.randn(n_samples) * 0.2
    base_data[:, 4] = base_data[:, 3] * 0.8 + np.random.randn(n_samples) * 0.2
    
    trait_df = pd.DataFrame(base_data, columns=trait_names)
    print(f"Generated synthetic data: {trait_df.shape}")

In [ ]:
# Display basic statistics
trait_df.describe()

## 4. Trait Correlation Analysis

In [ ]:
# Calculate trait correlations with p-values
corr_matrix, p_matrix = analyzer.calculate_trait_correlations(
    trait_df, 
    method='pearson'
)

# Visualize correlation matrix
fig = visualizer.plot_trait_correlation_heatmap(
    trait_df,
    method='pearson',
    figsize=(10, 8)
)
plt.show()

# Find significant correlations
significant_corrs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if p_matrix.iloc[i, j] < 0.05:
            significant_corrs.append({
                'Trait1': corr_matrix.columns[i],
                'Trait2': corr_matrix.columns[j],
                'Correlation': corr_matrix.iloc[i, j],
                'P-value': p_matrix.iloc[i, j]
            })

sig_corr_df = pd.DataFrame(significant_corrs).sort_values('P-value')
print(f"\nFound {len(sig_corr_df)} significant correlations (p < 0.05):")
sig_corr_df.head(10)

## 5. Interactive Correlation Heatmap

In [ ]:
# Create interactive heatmap
interactive_fig = visualizer.create_interactive_heatmap(trait_df, method='pearson')
interactive_fig.show()

## 6. Trait Distribution Analysis

In [ ]:
# Analyze distribution of each trait
n_cols = 3
n_rows = (len(trait_df.columns) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 4))
axes = axes.flatten()

for i, trait in enumerate(trait_df.columns):
    if i < len(axes):
        ax = axes[i]
        trait_df[trait].hist(bins=30, ax=ax, alpha=0.7, color='skyblue', edgecolor='black')
        ax.set_title(f'Distribution of {trait}')
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')
        
        # Add mean line
        mean_val = trait_df[trait].mean()
        ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_val:.2f}')
        ax.legend()

# Hide empty subplots
for i in range(len(trait_df.columns), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

## 7. PCA Analysis

In [ ]:
# Perform PCA
pca_results = analyzer.perform_pca_analysis(trait_df, n_components=5, standardize=True)

# Plot explained variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
ax1.plot(range(1, len(pca_results['explained_variance_ratio']) + 1), 
         pca_results['explained_variance_ratio'], 
         'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('Scree Plot')
ax1.grid(True, alpha=0.3)

# Cumulative variance explained
ax2.plot(range(1, len(pca_results['cumulative_variance_ratio']) + 1),
         pca_results['cumulative_variance_ratio'],
         'ro-', linewidth=2, markersize=8)
ax2.axhline(0.8, color='green', linestyle='--', label='80% variance')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Explained Variance')
ax2.set_title('Cumulative Variance Explained')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Variance explained by first 3 components: {pca_results['cumulative_variance_ratio'][2]:.2%}")

In [ ]:
# Plot PCA biplot
fig, ax = plt.subplots(figsize=(10, 8))

# Plot samples
pc_data = pca_results['transformed_data']
ax.scatter(pc_data['PC1'], pc_data['PC2'], alpha=0.5, s=30)

# Plot loadings as arrows
loadings = pca_results['loadings']
for trait in loadings.index:
    ax.arrow(0, 0, 
             loadings.loc[trait, 'PC1'] * 3, 
             loadings.loc[trait, 'PC2'] * 3,
             head_width=0.05, head_length=0.05, 
             fc='red', ec='red', alpha=0.7)
    ax.text(loadings.loc[trait, 'PC1'] * 3.2, 
            loadings.loc[trait, 'PC2'] * 3.2,
            trait, fontsize=10, ha='center', va='center')

ax.set_xlabel(f"PC1 ({pca_results['explained_variance_ratio'][0]:.1%} variance)")
ax.set_ylabel(f"PC2 ({pca_results['explained_variance_ratio'][1]:.1%} variance)")
ax.set_title('PCA Biplot')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Trait Clustering

In [ ]:
# Cluster traits based on their patterns
clustering_results = analyzer.cluster_traits(trait_df, method='kmeans', n_clusters=3)

# Display cluster assignments
cluster_df = pd.DataFrame({
    'Trait': list(clustering_results['trait_clusters'].keys()),
    'Cluster': list(clustering_results['trait_clusters'].values())
}).sort_values('Cluster')

print(f"Traits grouped into {clustering_results['n_clusters']} clusters:")
for cluster_id in range(clustering_results['n_clusters']):
    traits_in_cluster = cluster_df[cluster_df['Cluster'] == cluster_id]['Trait'].tolist()
    print(f"\nCluster {cluster_id}: {', '.join(traits_in_cluster)}")

## 9. Gene-Trait Association Analysis

In [ ]:
# Generate synthetic gene-trait associations for demonstration
# In real analysis, this would come from GWAS or other association studies
np.random.seed(42)

genes = [f"GENE{i}" for i in range(1, 21)]
gene_trait_associations = {}

for gene in genes:
    # Randomly assign traits to genes
    n_traits = np.random.choice([1, 2, 3, 4, 5], p=[0.3, 0.3, 0.2, 0.15, 0.05])
    associated_traits = np.random.choice(trait_df.columns, size=n_traits, replace=False).tolist()
    gene_trait_associations[gene] = associated_traits

# Calculate pleiotropy scores
pleiotropy_scores = analyzer.calculate_pleiotropy_score(
    gene_trait_associations,
    trait_correlations=corr_matrix,
    method='count_weighted'
)

# Visualize pleiotropy scores
fig = visualizer.plot_pleiotropy_score_distribution(
    pleiotropy_scores,
    threshold=2.5
)
plt.show()

## 10. Gene-Trait Network Visualization

In [ ]:
# Select top pleiotropic genes for visualization
sorted_genes = sorted(pleiotropy_scores.items(), key=lambda x: x[1], reverse=True)
top_genes = dict(sorted_genes[:10])
top_gene_associations = {gene: gene_trait_associations[gene] for gene in top_genes.keys()}

# Create network visualization
fig = visualizer.plot_gene_trait_network(
    top_gene_associations,
    layout='spring',
    node_size_factor=500
)
plt.show()

## 11. Interactive Sankey Diagram

In [ ]:
# Create Sankey diagram for gene-trait relationships
sankey_fig = visualizer.create_trait_gene_sankey(
    gene_trait_associations,
    top_n_genes=15
)
sankey_fig.show()

## 12. Export Results

In [ ]:
# Prepare results for export
results_dir = Path("./analysis_results")
results_dir.mkdir(exist_ok=True)

# Export correlation matrix
corr_matrix.to_csv(results_dir / "trait_correlations.csv")
print("Saved trait correlations")

# Export pleiotropy scores
pleiotropy_df = pd.DataFrame([
    {'gene': gene, 'pleiotropy_score': score, 'n_traits': len(gene_trait_associations[gene])}
    for gene, score in pleiotropy_scores.items()
]).sort_values('pleiotropy_score', ascending=False)

pleiotropy_df.to_csv(results_dir / "pleiotropy_scores.csv", index=False)
print("Saved pleiotropy scores")

# Export PCA results
pca_results['loadings'].to_csv(results_dir / "pca_loadings.csv")
pca_results['transformed_data'].to_csv(results_dir / "pca_scores.csv")
print("Saved PCA results")

# Create summary report
summary = {
    'n_samples': len(trait_df),
    'n_traits': len(trait_df.columns),
    'n_genes_analyzed': len(gene_trait_associations),
    'n_significant_correlations': len(sig_corr_df),
    'max_pleiotropy_score': max(pleiotropy_scores.values()),
    'variance_explained_3pc': float(pca_results['cumulative_variance_ratio'][2])
}

with open(results_dir / "analysis_summary.json", 'w') as f:
    import json
    json.dump(summary, f, indent=2)

print("\nAnalysis complete! Results saved to:", results_dir.absolute())

## 13. Custom Analysis Functions

In [ ]:
# Define custom analysis functions for your specific needs

def find_trait_pairs_with_high_correlation(corr_matrix, threshold=0.7):
    """Find pairs of traits with correlation above threshold."""
    high_corr_pairs = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr = corr_matrix.iloc[i, j]
            if abs(corr) > threshold:
                high_corr_pairs.append({
                    'trait1': corr_matrix.columns[i],
                    'trait2': corr_matrix.columns[j],
                    'correlation': corr
                })
    
    return pd.DataFrame(high_corr_pairs)

def identify_hub_genes(gene_trait_associations, min_traits=3):
    """Identify hub genes associated with many traits."""
    hub_genes = {
        gene: traits 
        for gene, traits in gene_trait_associations.items() 
        if len(traits) >= min_traits
    }
    return hub_genes

# Example usage
high_corr_pairs = find_trait_pairs_with_high_correlation(corr_matrix, threshold=0.5)
print(f"Found {len(high_corr_pairs)} trait pairs with |correlation| > 0.5")
high_corr_pairs.head()


In [ ]:
# Identify hub genes
hub_genes = identify_hub_genes(gene_trait_associations, min_traits=3)
print(f"Found {len(hub_genes)} hub genes associated with 3+ traits:")
for gene, traits in sorted(hub_genes.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"  {gene}: {len(traits)} traits - {', '.join(traits)}")

## Next Steps

This notebook provides a foundation for genomic pleiotropy analysis. You can extend it by:

1. **Loading real genomic data**: Replace synthetic data with actual GWAS results
2. **Adding pathway analysis**: Integrate pathway databases to understand biological functions
3. **Implementing advanced statistics**: Add more sophisticated statistical tests
4. **Creating custom visualizations**: Develop specialized plots for your specific research questions
5. **Building machine learning models**: Use the processed data for predictive modeling

Remember to save your work and document your findings!